In [1]:
%%spark

The code failed because of a fatal error:
	Invalid status code '500' from http://rapid-788.oit.duke.edu:8999/sessions with error payload: "java.net.ConnectException: Call From rapid-788.oit.duke.edu/67.159.92.217 to rapid-777.oit.duke.edu:8020 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused".

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
%%configure -f
{ "conf": {"spark.master": "yarn" },
"driverMemory": "4G", "numExecutors": 8, "executorCores": 2, 
"executorMemory": "1024M", "proxyUser": "user12026"}

In [ ]:
%%info

In [ ]:
%%help

In [ ]:
fname = '/data/bike_data/201408_babs_open_data/201408_status_data.csv'
df=spark.read.csv(fname, header='true', inferSchema='true')

In [ ]:
df.show(5)

## Movie Recommendation (ALS)

In [ ]:
import subprocess

cmd = 'hdfs dfs -ls /data/mllib/als'.split() 
files = subprocess.check_output(cmd).strip().split('\n')
for path in files:
    print (path)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
lines = spark.read.text("/data/mllib/als/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=long(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [ ]:
# using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
# Computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

In [ ]:
# top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

In [ ]:
# top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)